In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
# epochs = 50
epochs = 30
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/30
50000/50000 [==============================] - 499s 10ms/step - loss: 1.5502 - acc: 0.4255 - val_loss: 1.4147 - val_acc: 0.5118
Epoch 2/30
19072/50000 [==========>...................] - ETA: 4:42 - loss: 1.2033 - acc: 0.5661

50000/50000 [==============================] - 478s 10ms/step - loss: 1.1355 - acc: 0.5886 - val_loss: 1.8800 - val_acc: 0.4732
Epoch 3/30
45184/50000 [==========================>...] - ETA: 43s - loss: 0.9681 - acc: 0.6516

50000/50000 [==============================] - 472s 9ms/step - loss: 0.9625 - acc: 0.6544 - val_loss: 2.0496 - val_acc: 0.4729
Epoch 4/30
50000/50000 [==============================] - 473s 9ms/step - loss: 0.8655 - acc: 0.6905 - val_loss: 1.4974 - val_acc: 0.5819
Epoch 5/30
 3200/50000 [>.............................] - ETA: 7:52 - loss: 0.7777 - acc: 0.7134

50000/50000 [==============================] - 475s 9ms/step - loss: 0.7887 - acc: 0.7184 - val_loss: 1.3500 - val_acc: 0.6153
Epoch 6/30
39296/50000 [======================>.......] - ETA: 1:36 - loss: 0.7364 - acc: 0.7405

50000/50000 [==============================] - 477s 10ms/step - loss: 0.7365 - acc: 0.7406 - val_loss: 1.5039 - val_acc: 0.6128
Epoch 7/30
50000/50000 [==============================] - 477s 10ms/step - loss: 0.6819 - acc: 0.7607 - val_loss: 1.5251 - val_acc: 0.6090
Epoch 8/30
 1664/50000 [..............................] - ETA: 8:17 - loss: 0.6283 - acc: 0.7770

...

50000/50000 [==============================] - 257s 5ms/step - loss: 0.3591 - acc: 0.8715 - val_loss: 0.5682 - val_acc: 0.8264
Epoch 25/30
50000/50000 [==============================] - 257s 5ms/step - loss: 0.3474 - acc: 0.8775 - val_loss: 0.9012 - val_acc: 0.7540
Epoch 26/30
 1536/50000 [..............................] - ETA: 3:53 - loss: 0.3453 - acc: 0.8770

50000/50000 [==============================] - 256s 5ms/step - loss: 0.3357 - acc: 0.8821 - val_loss: 0.9121 - val_acc: 0.7583
Epoch 27/30
38528/50000 [======================>.......] - ETA: 55s - loss: 0.3290 - acc: 0.8853

50000/50000 [==============================] - 257s 5ms/step - loss: 0.3288 - acc: 0.8849 - val_loss: 0.6996 - val_acc: 0.8014
Epoch 28/30
50000/50000 [==============================] - 257s 5ms/step - loss: 0.3244 - acc: 0.8866 - val_loss: 0.9402 - val_acc: 0.7632
Epoch 29/30
 1536/50000 [..............................] - ETA: 3:54 - loss: 0.2631 - acc: 0.9023

50000/50000 [==============================] - 257s 5ms/step - loss: 0.3128 - acc: 0.8880 - val_loss: 0.8191 - val_acc: 0.7848
Epoch 30/30
38528/50000 [======================>.......] - ETA: 55s - loss: 0.3095 - acc: 0.8908

50000/50000 [==============================] - 257s 5ms/step - loss: 0.3107 - acc: 0.8908 - val_loss: 0.6596 - val_acc: 0.8203


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 22s 2ms/step
Test loss: 0.6596327358245849
Test accuracy: 0.8203


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')